In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
from sklearn.metrics import r2_score
from mlxtend.evaluate import PredefinedHoldoutSplit


c:\Users\suvam\miniconda3\envs\aie2021\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


<IPython.core.display.Javascript object>

In [2]:
%run functions.ipynb

In [3]:
station = "erlabrunn"
flow_station = "erlabrunn"
start = "2016-11-15"
end = "2020-12-30"

In [4]:
#OPTIONS:  "OPO4P", "NO3N", "NH4N", "TRP"
predict = "OPO4P"
# OPTIONS: "lr", "rf", 
test_model = "rf"

results = pd.DataFrame([1])

In [5]:
if test_model == "rf":
    algo = RandomForestRegressor(n_jobs = -1)
elif test_model == "lr":
    algo = linear_model.LinearRegression()

In [7]:
df = pd.read_csv(r'./data/original_data/all_data_%s.csv' %(station))
df['Datum'] = pd.to_datetime(df['Datum'])
df= df.set_index("Datum")
df = df.round(2)
# df

In [8]:
flow = pd.read_csv("./data/original_data/flow_%s.csv" %(flow_station))
flow['Datum'] = pd.to_datetime(flow['Datum'])
flow= flow.set_index("Datum")
flow = flow.loc[start:end]
df = pd.concat([df, flow], axis=1)
df = df.dropna(thresh = 3)

In [9]:
if station == "kahl":
    if predict == "OPO4P" or predict == "NH4N":
        df = df.loc["2019-09-02":]

In [10]:
df.describe()

,O2,Temp,Conduct,pH,OPO4P,NO3N,flow
count,136403.000000,135616.000000,135767.000000,133741.000000,130146.000000,135839.000000,27519.000000
mean,10.138804,13.208772,637.663208,7.942306,0.102662,3.931959,201.956957
std,2.143421,7.517295,83.779758,0.207624,0.082252,1.030602,89.229159
min,5.300000,0.100000,356.000000,7.300000,0.000000,1.500000,39.800000
25%,8.400000,6.100000,590.000000,7.800000,0.080000,3.100000,152.000000
50%,10.100000,12.700000,654.000000,7.900000,0.110000,4.000000,191.000000
75%,12.200000,20.400000,698.000000,8.100000,0.120000,4.700000,246.000000
max,19.100000,28.800000,842.000000,8.600000,3.040000,6.850000,627.000000


In [11]:
Pre_count = count(df, "Pre Count")

df = df[["O2", "Temp", "Conduct", "pH", "flow", predict]]         
df = df.dropna(subset=[predict])

Post_count = count(df, "Post Count")

columns = df.columns

In [12]:
Final_data = Data_count(Pre_count, Post_count)
print(station)
Final_data.dropna()

erlabrunn


,Pre Count,Post Count,Loss,Percentage Loss
Parameter,,,,
O2,136403.0,130047.0,6356.0,4.659722
Temp,135616.0,129258.0,6358.0,4.688237
Conduct,135767.0,129412.0,6355.0,4.680813
pH,133741.0,128169.0,5572.0,4.166262
OPO4P,130146.0,130146.0,0.0,0.000000
flow,27519.0,26793.0,726.0,2.638177


### Cleaning

In [13]:
#ONLY FOR ERLABRUNN TO CLEAR PHOSPHATE A BIT
if station == "erlabrunn":
    if predict == "OPO4P":
        para = "OPO4P"
        temp = df[[para]]
        df= df.drop([para],axis =1)
        temp = temp[(temp[para] < 0.3)]   
        df = pd.concat([df, temp], axis=1)
        df = df.loc["2019-01-01 00:00:00":]
        
            
    if predict == "NH4N":
        df['NH4N'].mask(df['NH4N'].between(-0.8, 0.001), inplace=True)    

In [14]:
if station == "kahl":
    # Conduct cleaning
    df.loc["2017-11-16": "2017-11-22", "Conduct"] = np.nan
    df['Conduct'].mask(df['Conduct'].between(0, 350), inplace=True)
    
    #Temp Cleaning
    df.loc["2017-11-17": "2017-11-21", "Temp"] = np.nan
    df.loc["2019-05-21 09:30:00": "2019-05-23 23:45:00", "Temp"] = np.nan
    
    #O2 Cleaning
    df.loc["2017-11-16 09:15:00": "2017-11-16 23:45:00", "O2"] = np.nan
    df.loc["2019-05-21 09:30:00": "2019-05-23 23:45:00", "O2"] = np.nan
    
    #NO3N cleaning
    if predict == "NO3N":
        df['NO3N'].mask(df['NO3N'].between(-1, 2), inplace=True)
        df.loc["2021-08-27 08:30:00", "NO3N"] = np.nan
    
    #NH4N cleaning
    if predict == "NH4N":
        df['NH4N'].mask(df['NH4N'].between(0.5, 40), inplace=True)
        df['NH4N'].mask(df['NH4N'].between(-0.8, 0.001), inplace=True)
    
    #OPO4P cleaning
    if predict == "OPO4P":
        df['OPO4P'].mask(df['OPO4P'].between(0.25, 5), inplace=True)
        df['OPO4P'].mask(df['OPO4P'].between(-0.8, 0.001), inplace=True)    

##### add removal , remove anomaly rows and create a dataframe

### Interpolation

In [16]:
bef_interpol = df
df = df.interpolate(limit = 30)
df = df.dropna()
df = df.round(2)
# df.to_csv(r'%s_cleaned_interpolated.csv' %station, index = True, header = True)
#     print(df)

In [17]:
removal = 0.1
from sklearn.ensemble import IsolationForest
clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(removal), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(df)
pred = clf.predict(df)
df['anomaly']=pred
outliers=df.loc[df['anomaly']==-1]
outlier_index=list(outliers.index)
#print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(df['anomaly'].value_counts())

df=df.loc[df['anomaly']==1]
df= df.drop(["anomaly"],axis =1)

anomaly
 1    8846
-1     983
Name: count, dtype: int64


### Time as a feature

In [18]:
df = df.reset_index()
df["Month"] = df['Datum'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

df['week_number'] = df["Datum"].dt.isocalendar().week
df["week_sin"] = np.sin(2 * np.pi * df["week_number"] / df["week_number"].max())
df["week_cos"] = np.cos(2 * np.pi * df["week_number"] / df["week_number"].max())
df= df.drop(["Month", "week_number"],axis =1)
df = df.set_index("Datum")

### Correlation Analysis

In [20]:
columns=df.columns
corr = df.corr()
corr = corr.loc[:,[predict]]
corr = corr.iloc[(-corr[predict].abs()).argsort()]
if predict == "NO3N":
    corr= corr.drop(["NO3N"],axis =0)

if predict == "OPO4P":
    corr= corr.drop(["OPO4P"],axis =0)
    
if predict == "NH4N":
    corr= corr.drop(["NH4N"],axis =0)
    
if predict == "TRP":
    corr= corr.drop(["TRP", "NO3N"],axis =0)
print(corr)
corr_order = list(corr.index)

              OPO4P
flow       0.249693
pH         0.178631
Conduct   -0.133574
week_sin  -0.076316
Temp       0.073824
O2         0.054421
month_sin -0.047108
anomaly   -0.040862
month_cos -0.006112
week_cos  -0.001641


In [25]:
df.to_csv(rf".\data\cleaned_up_data\{station}_{predict}_final.csv")